In [ ]:

import holoviews as hv
import panel as pn
import numpy as np
import pandas as pd
import hvplot.pandas
df = pd.read_csv("./data/stations_start.csv")

def test():
    df = pd.read_csv("./data/stations_start.csv")
    def map():
        return df.hvplot.points("lon", "lat", geo=True, hover_cols='Name',tiles=True, xaxis=None, yaxis=None, responsive=True)
    return pn.Column(
        pn.pane.HoloViews(hv.DynamicMap(map), sizing_mode="stretch_both"),
                sizing_mode="stretch_both",
    )
test()

In [ ]:
import param

class Map(param.Parameterized):
    data = pd.read_csv("./data/stations_start.csv")
    value = param.Integer(default=10, bounds=(1, 10000))
    
    @param.depends('value')
    def create_map(self):
        df = self.data.loc[self.data["value"] >= self.value]
        return df.hvplot.points('lon', 'lat', geo=True, tiles=True)
    
map = Map()
map_dmap = hv.DynamicMap(map.create_map)
pn.Row(map.param, map_dmap)



In [ ]:
"""def _otherwaytodoit(data,title, x, widget, xlabel):
    def hooks(plot, element):
        p = plot.state
        p.toolbar.autohide = True
        p.toolbar.logo = None
        if "trips" in widget.value:
            p.yaxis.ticker=[0,2000000,4000000,6000000,8000000]
            p.yaxis.major_label_overrides={0:"0",2000000:"2M",4000000:"4M",6000000:"6M",8000000:"8M"} 
        else:
            p.yaxis.ticker = np.arange(0, 1400, 200)
            p.yaxis.major_label.overrides={0:"0", 200:"200", 400:"400", 600:"600", 800:"800", 1000:"1000",1200:"1200", 1400:"1400"}
    
    def plot(widget = widget):
        return data.hvplot(kind="bar",
                           x=x,
                           title=title,
                           xlabel=xlabel,
                           ylabel=re.sub(r"\([a-z,A-Z]*\)","", str(widget)).capitalize(),
                           y=widget,
                           color=MAIN,
                           line_color="white",
                           responsive=True,
                           min_width=350,
                           min_height=250,
                           label=re.sub(r"\([a-z,A-Z]*\)","", str(widget)).capitalize()).opts(legend_position='top',hooks=[hooks]) \
            * data.hvplot.line(x=x,
                             y=f"mean{widget}",
                             color=ACCENT,
                             responsive=True,
                             line_color="white",
                             min_width=350,
                             min_height=250,
                             label="Mean")
    plot = pn.bind(plot, widget=widget)
    return pn.Column(
        pn.pane.HoloViews(hv.DynamicMap(plot), sizing_mode="stretch_both"),
                sizing_mode="stretch_both",
    )"""
    
"""def _create_geocoord():
docks = pd.read_csv("./data/coordinates.csv")
docks["geometry"] = "POINT (" + docks["lon"].astype(str) + " " + docks["lat"].astype(str) + ")"
docks = docks[["station", "geometry"]]
docks.rename(columns={"station": "name"}, inplace = True)
docks.to_csv("./coordinatesGeo.csv", index=False)"""

"""def createGeodata():
    df4 = pd.read_csv("./data/popular_journeys.csv", sep=",")
    top = df4.sort_values("pop_journeys_value", ascending=False)
    coord = pd.read_csv("./coordinates.csv")
    top10 = top.head(10)
    top10 = pd.merge(top10, coord, on=["station"])
    top10.rename(columns={"station":"start_station", "lat": "start_lat", "lon": "start_lon", "end_station_name": "station"}, inplace = True)
    top10 = pd.merge(top10, coord, on=["station"])
    top10.rename(columns={"station":"end_station","lat":"end_lat", "lon":"end_lon"}, inplace=True)
    top10.drop(["stationid_x", "stationid_y"],axis=1, inplace = True)
    top10
    #   "LINESTRING (-3.7038 40.4168,38 56)","LineString"
    top10["geometry"] = "LINESTRING (" + top10["start_lon"].astype(str) + " " + top10["start_lat"].astype(str) + "," + top10["end_lon"].astype(str) + " " + top10["end_lat"].astype(str) + ")"
    top10["name"] = top10["start_station"] + " to " + top10["end_station"]
    top10Geo = top10[["name", "geometry"]]
    top10Geo.to_csv("./data/top10Geo.csv", index=False)"""

In [9]:
import param
import holoviews as hv
import panel as pn
import numpy as np
import pandas as pd
import geopandas as gpd
import re
from panel.template import FastGridTemplate
from panel.template import DarkTheme
import hvplot.pandas
 
MAIN = "#D6ADD0"
ACCENT = "#724175"
df1 = pd.read_csv("./data/trips_by_month.csv", sep=",")
df2 = pd.read_csv("./data/trips_by_year.csv", sep=",")
df3 = pd.read_csv("./data/trips_by_hour.csv", sep=",")
df3["mean"] = df3["trips"].mean()

In [ ]:

def _create_barandlinewidget(data, x, title="Title"):
    
    class Plot(param.Parameterized):
        
        column = param.ObjectSelector(default=f"trips({x})", objects=[f"duration({x})", f"trips({x})"])
        
        @param.depends("column")
        def create_plot(self):
            
            def hooks(plot, element):
                print(self.column)
                p = plot.state
                p.toolbar.autohide = True
                p.toolbar.logo = None
                if "trips" in self.column:
                    p.yaxis.ticker=[0,2000000,4000000,6000000,8000000]
                    p.yaxis.major_label_overrides={0:"0",2000000:"2M",4000000:"4M",6000000:"6M",8000000:"8M"} 
                else:
                    p.yaxis.ticker = np.arange(0, 1400, 200)
                    p.yaxis.major_label.overrides={0:"0", 200:"200", 400:"400", 600:"600", 800:"800", 1000:"1000",1200:"1200", 1400:"1400"}
    
                
            return data.hvplot(kind="bar",
                            x=x,
                            title=title,
                            y=self.column,
                            color=MAIN,
                            xlabel=str(x).capitalize(),
                            line_color="white",
                            responsive=True,
                            min_width=350,
                            min_height=250,
                            ylabel=re.sub(r"\([a-z,A-Z]*\)","", str(self.column)).capitalize(),
                            label=re.sub(r"\([a-z,A-Z]*\)","", str(self.column)).capitalize()).opts(legend_position='top', hooks=[hooks]) \
                                * data.hvplot(kind="line",
                                            x=x,
                                            y=f"mean{self.column}",
                                            responsive=True, 
                                            line_color="white",
                                            label="Mean")
        @param.depends("column")
        def view(self):
            plot = hv.DynamicMap(self.create_plot)
            return plot
        
    p = Plot()
    return pn.Column(
        pn.Param(p.param, name="Foo"),
        p.view, sizing_mode="stretch_both"
    )
    
    
app =pn.Column(_create_barandlinewidget(data=df1, x="month"),
               _create_barandlinewidget(data=df2, x="year"))
app.show()

In [12]:
def _create_barandline(data, x, title="Title"):
    
    class Plot(param.Parameterized):        
        def create_plot(self):
            
            def hooks(plot, element):
                p = plot.state
                p.toolbar.autohide = True
                p.toolbar.logo = None
                p.yaxis.ticker=[0,2000000,4000000,6000000,8000000]
                p.yaxis.major_label_overrides={0:"0",2000000:"2M",4000000:"4M",6000000:"6M",8000000:"8M"} 
                
            return data.hvplot(kind="bar",
                            x=x,
                            title=title,
                            y="title",
                            color=MAIN,
                            xlabel=str(x).capitalize(),
                            line_color="white",
                            responsive=True,
                            min_width=350,
                            min_height=250,
                            ylabel=str(y).capitalize(),
                            label=str(y).capitalize()).opts(legend_position='right', hooks=[hooks]) \
                                * data.hvplot(kind="line",
                                            x=x,
                                            y="mean",
                                            responsive=True, 
                                            line_color="white",
                                            label="Mean")
        def view(self):
            plot = hv.DynamicMap(self.create_plot)
            return plot
        
    p = Plot()
    return pn.Column(p.view, sizing_mode="stretch_both")


Column(sizing_mode='stretch_both')
    [0] ParamMethod(method)

In [42]:
def test(data, x, title="Title"):
    
    class Plot(param.Parameterized):
        
        select_your_metric = param.ObjectSelector(default=f"trips({x})", objects=[f"duration({x})", f"trips({x})"])
        
        @param.depends("select_your_metric")
        def create_plot(self):
            
            def hooks(plot, element):
                p = plot.state
                p.toolbar.autohide = True
                p.toolbar.logo = None
                if "trips" in self.select_your_metric:
                    p.yaxis.ticker=[0,2000000,4000000,6000000,8000000]
                    p.yaxis.major_label_overrides={0:"0",2000000:"2M",4000000:"4M",6000000:"6M",8000000:"8M"} 
                else:
                    p.yaxis.ticker = np.arange(0, 1400, 200)
                    p.yaxis.major_label.overrides={0:"0", 200:"200", 400:"400", 600:"600", 800:"800", 1000:"1000",1200:"1200", 1400:"1400"}
    
                
            return data.hvplot(kind="bar",
                            x=x,
                            title=title,
                            y=self.select_your_metric,
                            color="#faceff",
                            xlabel=str(x).capitalize(),
                            line_color="white",
                            responsive=True,
                            min_width=350,
                            min_height=250,
                            ylabel=re.sub(r"\([a-z,A-Z]*\)","", str(self.select_your_metric)).capitalize(),
                            label=re.sub(r"\([a-z,A-Z]*\)","", str(self.select_your_metric)).capitalize()).opts(legend_position='top', hooks=[hooks]) \
                                * data.hvplot(kind="line",
                                            x=x,
                                            y=f"mean{self.select_your_metric}",
                                            responsive=True, 
                                            line_color="white",
                                            label="Mean")
        @param.depends("select_your_metric")
        def view(self):
            plot = hv.DynamicMap(self.create_plot)
            return plot
        
    p = Plot()
    return pn.Column(
            pn.Row(
            pn.layout.VSpacer(width=500),
            pn.Param(p.param,
                 show_name=False,
                 show_labels = False,               
                 )),
            pn.Row(
        p.view, sizing_mode="stretch_both"
    ))


Column
    [0] Row
        [0] VSpacer(width=500)
        [1] Param(Plot, show_labels=False, show_name=False)
    [1] Row(sizing_mode='stretch_both')
        [0] ParamMethod(method)